# Constrained thrust allocation for a marine vessel with 4 rotatable/azimuth thrusters
The problem of control allocation, or in our case thrust allocation, is the common problem of distributing a "total" control effort (usually a force vector) among multiple actuators. In this example we want to solve the thrust allocation problem for a surface vessel(boat) with 4 rotatable thrusters. The task of the algorithm will be to take a desired force vector (longitudal, lateral and angular) as input, and calculate the force and angle for each individual thruster.

Since we have four thrusters, but only three degrees of freedom, our system is "overactuated". This means that there are infinitely many solutions for individual thruster forces and angles which result in the desired total forces. In addition, we have constraints on how much force each thruster can produce (the physical limits of the thrusters), which must be considered.

First, a quadratic programming approach is taken, where the problem is formulated as a quadratic cost function, which punishes the deviation between desired force and achieved force, and the total "effort" expended by the thrusters. The physical limit of the thrusters is also included as a hard constraint for the solver to handle.

Then, the problem is solved using the moore-penrose psuedo-inverse, which given an underdetermined system, like our "overactuated" one, will find the "smallest" solution, in the L2 norm sense. This approac is less flexible, contraints are harder to enforce, but is *much* faster, as each solution is essentially a single matrix multiplication.

## quadratic programming approach
We want to achieve the generalized desired thrust $$[F_x, Fy, Mz]^T = \vec{\tau} = B\vec{u}$$
By choosing a minimal control input $\vec{u}$ such that we achieve the desired thrust with minimal thruster effort.

The thrust vector is expressed in cartesian coordinates, as 4 pairs of thrusters:
$$ \vec{u_k} = \begin{bmatrix} u_{kx} & u_{ky} \end{bmatrix}^T $$
$$ \vec{u} = \begin{bmatrix} \vec{u_1} \\ \vec{u_2} \\ \vec{u_3} \\ \vec{u_4} \end{bmatrix} $$

The B matrix is the thruster configuration matrix, which maps control inputs (thruster forces) to forces/moments on the boat itself.

A simple way to reduce effort/power consumption is to find a solution which minimizes the sum of the squares of the individual truster components:
$$\sum_{i=1}^4 {u}^2_{ix} + {u}^2_{iy} $$

One way to solve this problem is to express it as a quadratic programming problem, where we try to minimize a quadratic cost function, subject to certain constraints.
One formulation of this problem is:

$$\min_{\vec{u}} \vec{u}^T \vec{u}$$
$$\textrm{subject to: } \vec{\tau} = B\vec{u}$$

However, this formulation is not very flexible, and may not ensure that we find a solution.
To deal with this, we introduce a slack variable $\vec{s} = \begin{bmatrix} s_1 \\ s_2 \\ s_3\end{bmatrix} $

With this slack variable, we can reformulate our strict constraint to a soft constraint:
$$ \vec{\tau} - B\vec{u} = \vec{s} $$
To ensure that we find a solution $\vec{u}$ which adequately approximates the equation $\vec{\tau} = B\vec{u}$, we include the slack variable into our cost function, and weight it heavily. Finally, we also add a constraint such that none of the thrusters exceed their maximum thruster limits.

The new formulation:
$$\min_{\vec{u}} \vec{x}^T P\vec{x}$$
$$\textrm{subject to: } A\vec{x} = \vec{\tau}, \quad ||u_k|| < u_{max} $$

<center> 

Where: $$\vec{x} = \begin{bmatrix} \vec{u} \\ \vec{s} \end{bmatrix}, \quad A = \begin{bmatrix} B & I_{3x3} \end{bmatrix}$$


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
import cvxpy as cp

#defining the thruster configuration matrix B (contains the positions of the thrusters)
l1_x = -1
l1_y = -0.6
l2_x = -1
l2_y = 0.6
l3_x = 1
l3_y = -0.6
l4_x = 1
l4_y = 0.6
thruster_max = 2
t_d = np.array([5, 0, 0])  # example desired generalized force
B = np.matrix(np.array([[  1,     0,     1,     0,     1,     0,     1,     0  ],  # thruster configuration matrix
                        [  0,     1,     0,     1,     0,     1,     0,     1  ],
                        [-l1_y,  l1_x, -l2_y,  l2_x, -l3_y,  l3_x, -l4_y,  l4_x]]))


A = np.array(np.hstack((B, np.eye(3))))  # A = [B, I]
tau_ = cp.Parameter(3) # desired generalized force
x = cp.Variable(11) # x = [u ; s]
#constructing the weight matrix, weighting s heavily to ensure a good approximation of the solution
P = np.array(np.vstack((np.hstack((np.eye(8), np.zeros((8, 3)))), np.hstack((np.zeros((3, 8)), 10000 * np.eye(3))))))

tau_.value = t_d # set the desired generalized force for the solver
constraints = [A @ x == tau_] #add the reformulated equality constraint including slack variables

#adding the constraints on maximum thruster force
for i in [0, 2, 4, 6]:
    constraints.append(x[i]**2 + x[i+1]**2 <= thruster_max)


prob = cp.Problem(cp.Minimize(cp.quad_form(x, P)), constraints) #define the optimization problem
prob.solve()



ValueError: Incompatible dimensions (3, 9) (11, 1)

In [2]:
def plot_fcn(Fx=0, Fy=0, Mz=0):
    tau_.value = np.array([Fx, Fy, Mz])
    prob.solve()

    #individual thruster forces
    u = x.value[0:8]
    u_1 = u[0:2]
    u_2 = u[2:4]
    u_3 = u[4:6]
    u_4 = u[6:8]
    #u = u_1 + u_2 + u_3 + u_4

    plt.arrow(l1_y, l1_x, u_1[1], u_1[0], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l2_y, l2_x, u_2[1], u_2[0], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l3_y, l3_x, u_3[1], u_3[0], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l4_y, l4_x, u_4[1], u_4[0], head_width=0.1, head_length=0.1, fc='k', ec='k')

    plt.arrow(0, 0, u[1]/4, u[0]/4, head_width=0.1, head_length=0.1, fc='r', ec='r')
    #limit axes
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)

    plt.grid()
    plt.plot()



In [3]:
ipywidgets.interact(plot_fcn, Fx=(-7, 7, 0.1), Fy=(-7, 7, 0.1), Mz=(-7, 7, 0.1))

interactive(children=(FloatSlider(value=0.0, description='Fx', max=7.0, min=-7.0), FloatSlider(value=0.0, desc…

<function __main__.plot_fcn(Fx=0, Fy=0, Mz=0)>

## psuedo-inverse approach
Now, let's try to solve the same problem using the psuedo-inverse.
We want to find a solution to the problem:
$$\vec{\tau} = B\vec{u} $$
In a way that gives us the smallest control effort $\vec{u}$.
One way to do this is to minimize the "norm" of u. Specifically, we will minimize the function:
$$\sum_i {u_i}^2$$
Solving this problem will minimize the sum of the vector components, which will hopefully minimize power consumption.
An explicit solution to this least-squares problem is given by the moore-penrose psuedo-inverse.

Specifically, the solution to $\vec{\tau} = B\vec{u}$ which minimizes the norm of $\vec{u}$, is given by:
$$B^+\vec{\tau}$$ 
Where $B^+$ denotes the psuedo-inverse of B.

The psuedo-inverse of B can be computed via the singular value decomposition(SVD) of B:
$$B = U\Sigma V^T$$
$$B^+ = V{\Sigma}^{-1} U^T$$

Or, by simply using numpy's pinv metod. 

To enforce the constraint on maximum thruster force, we can simply check whether the psuedo-inverse solution violates the constraint, then normalize the thruster forces to avoid the violation.

This approach is much faster than the QP formulation, as the we only have to compute the SVD once, and perform a single matrix multiplication and a few conditionals for each control allocation. The downside of the approach is that it's less flexible to more complicated constraints and considerations.

In [4]:
#compute the psuedo-inverse of B using the numpy pinv
B_pinv = np.linalg.pinv(B)
tau = np.array([[10],[10],[0]])
u = B_pinv @ tau

In [5]:
def plot_fcn_pinv(Fx=0, Fy=0, Mz=0):
    tau = np.hstack([Fx, Fy, Mz])
    u = B_pinv @ tau

    #calculate the thrust pair forces
    f = np.zeros(4)
    for i in range(4):
        f[i] = np.linalg.norm(u[2*i:2*i+2])
    if np.max(f) > thruster_max:
        u = u * thruster_max / np.max(f)

    u_1 = u[0, 0:2]
    u_2 = u[0, 2:4]
    u_3 = u[0, 4:6]
    u_4 = u[0, 6:8]


    u = u_1 + u_2 + u_3 + u_4

    plt.arrow(l1_y, l1_x, u_1[0, 1], u_1[0, 0], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l2_y, l2_x, u_2[0, 1], u_2[0, 0], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l3_y, l3_x, u_3[0, 1], u_3[0, 0], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l4_y, l4_x, u_4[0, 1], u_4[0, 0], head_width=0.1, head_length=0.1, fc='k', ec='k')

    plt.arrow(0, 0, u[0, 1]/4, u[0, 0]/4, head_width=0.1, head_length=0.1, fc='r', ec='r')
    #limit axes
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)

    plt.grid()
    plt.plot()


In [6]:
ipywidgets.interact(plot_fcn_pinv, Fx=(-7, 7, 0.1), Fy=(-7, 7, 0.1), Mz=(-7, 7, 0.1))

interactive(children=(FloatSlider(value=0.0, description='Fx', max=7.0, min=-7.0), FloatSlider(value=0.0, desc…

<function __main__.plot_fcn_pinv(Fx=0, Fy=0, Mz=0)>

### Fixed thruster configuration (X + Bow)

In [81]:
#rear left
L1_X = -2.373776 
L1_Y = 1.027135
D_1 = np.sqrt(L1_X**2 + L1_Y**2)
#rear right
L2_X = -2.373776 
L2_Y = -1.027135
D_2 = np.sqrt(L2_X**2 + L2_Y**2)
#front left
L3_X = 1.75
L3_Y = 1.027135
D_3 = np.sqrt(L3_X**2 + L3_Y**2)
#front right
L4_X = 1.75
L4_Y = -1.027135
D_4 = np.sqrt(L4_X**2 + L4_Y**2)

THRUST_LIMIT = 200
ANGLE_LIMIT = np.pi*0.8 #bi-directional
theta = np.pi/4

B_x = np.matrix(np.array([[ np.cos(theta),       np.cos(theta),        0,     0  ],  # thruster configuration matrix
                          [ np.sin(theta),      -np.sin(theta),       1,    -1  ],
                          [-D_1*np.sin(theta),   D_2*np.sin(theta),  L3_X,  -L4_X ]]))
B_pinv_x = np.linalg.pinv(B_x)

In [82]:
def plot_fcn_pinv_X(Fx=0, Fy=0, Mz=0):
    tau = np.hstack([Fx, Fy, Mz])
    u = B_pinv_x @ tau

    plt.arrow(l1_y, l1_x, u[0,0]*np.cos(np.pi/4), u[0,0]*np.cos(np.pi/4), head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l2_y, l2_x, -u[0,1]*np.sin(np.pi/4), u[0,1]*np.cos(np.pi/4), head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l3_y, l3_x, 0, u[0,2], head_width=0.1, head_length=0.1, fc='k', ec='k')
    plt.arrow(l4_y, l4_x, 0, u[0,3], head_width=0.1, head_length=0.1, fc='k', ec='k')

    #limit axes
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)

    plt.grid()
    plt.plot()


In [71]:
tau = np.array([[0],[10],[0]])
u = B_pinv_x @ tau
u


matrix([[ 7.07106781e+00],
        [-7.07106781e+00],
        [ 3.08338684e-15],
        [-1.63587694e-15]])

In [72]:
ipywidgets.interact(plot_fcn_pinv_X, Fx=(-7, 7, 0.1), Fy=(-7, 7, 0.1), Mz=(-7, 7, 0.1))


interactive(children=(FloatSlider(value=0.0, description='Fx', max=7.0, min=-7.0), FloatSlider(value=0.0, desc…

<function __main__.plot_fcn_pinv_X(Fx=0, Fy=0, Mz=0)>